In [1]:
!pip install pyterrier
!pip install python-terrier
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
import pyterrier as pt
if not pt.started():
    pt.init()

<ipython-input-3-a9775fa13689>:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


terrier-assemblies 5.10 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
<ipython-input-3-a9775fa13689>:3: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [4]:
df_q_explor= pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/query_proc2005_agr.tsv", sep='\t')
df_q_explor['qid'] = df_q_explor['qid'].astype(str)
print(len(df_q_explor.index))

19


In [5]:
print(len(df_q_explor.index))

19


In [6]:
# remove line breaks and extra spaces
def clean_query(query):
    return ' '.join(query.replace('\n', ' ').split())
df_q_explor['query'] = df_q_explor['query'].apply(clean_query)

In [7]:
df_q_explor

,qid,query,intent_fin
0,203,code signing verification,process
1,209,mining frequent pattern itemset sequence graph...,process
2,212,hmm hidden markov model equation,process
3,218,computer assisted composing music notes midi,process
4,221,capabilities limitations commercial speech rec...,process
5,241,single sign on ldap,process
6,242,web personalization approaches,process
7,248,clustering,process
8,249,clustering,process
9,253,evaluation usability experiment digital librar...,process


In [9]:
index = pt.IndexFactory.of("/content/drive/MyDrive/data_colab/inex_index/sections_no_sw_2005")

In [10]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=1500)


bm25_res = bm25.transform(df_q_explor)


# Retrieve the TF-IDF scores
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", num_results=1500)
tfidf_res = tfidf.transform(df_q_explor)
tfidf_res


combined_results = pd.merge(bm25_res, tfidf_res[['qid', 'docid', 'score']], on=['qid', 'docid'], suffixes=('_bm25', '_tfidf'))

combined_results

<ipython-input-10-d0fa344fec9c>:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=1500)
<ipython-input-10-d0fa344fec9c>:12: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", num_results=1500)


,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf
0,203,23601,co/2000/r7108/article[1]/bdy[1]/sec[2],0,22.160864,code signing verification,process,12.755553
1,203,11887,dt/2001/d4006/article[1]/bdy[1]/sec[3],1,21.497232,code signing verification,process,12.350038
2,203,11888,dt/2001/d4006/article[1]/bdy[1]/sec[4],2,21.165475,code signing verification,process,12.153532
3,203,24890,co/1997/r6076/article[1]/bdy[1]/sec[4],3,20.580498,code signing verification,process,11.867551
4,203,25503,co/1997/r6026/article[1]/bdy[1]/sec[6],4,20.172168,code signing verification,process,11.672510
...,...,...,...,...,...,...,...,...
23618,285,24430,co/2000/r5012/article[1]/bdy[1]/sec[1],1491,2.280486,operating system,process,3.790378
23619,285,48529,tk/1997/k0600/article[1]/bdy[1]/sec[5],1492,2.280477,operating system,process,3.689152
23620,285,63709,td/1999/l0076/article[1]/bdy[1]/sec[1],1494,2.279939,operating system,process,4.025588
23621,285,18209,cg/1998/g4030/article[1]/bdy[1]/sec[1],1495,2.279900,operating system,process,3.670078


In [ ]:
import pandas as pd

df_th = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/thorough_no_empty_no_sw05_corr.tsv',sep="\t")
combined_results = pd.merge(combined_results, df_th[['docno', 'text']], on='docno', how='left')
combined_results

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text
0,203,23601,co/2000/r7108/article[1]/bdy[1]/sec[2],0,22.160864,code signing verification,process,12.755553,"CODE SIGNING VERIFICATION Typically, software ..."
1,203,11887,dt/2001/d4006/article[1]/bdy[1]/sec[3],1,21.497232,code signing verification,process,12.350038,RTL STATIC SIGN-OFF ISSUES Using concepts fram...
2,203,11888,dt/2001/d4006/article[1]/bdy[1]/sec[4],2,21.165475,code signing verification,process,12.153532,EFFECTIVE EQUIVALENCE-CHECKING METHODOLOGY Tod...
3,203,24890,co/1997/r6076/article[1]/bdy[1]/sec[4],3,20.580498,code signing verification,process,11.867551,CONCLUSION general-purpose secure code distrib...
4,203,25503,co/1997/r6026/article[1]/bdy[1]/sec[6],4,20.172168,code signing verification,process,11.672510,"Secure Code Distribution, pp. 76-79 X. Nick Zh..."
...,...,...,...,...,...,...,...,...,...
23618,285,24430,co/2000/r5012/article[1]/bdy[1]/sec[1],1491,2.280486,operating system,process,3.790378,"companies adopt e-commerce, extranets, intrane..."
23619,285,48529,tk/1997/k0600/article[1]/bdy[1]/sec[5],1492,2.280477,operating system,process,3.689152,MULTIVIEW: VIEW SYSTEM SUPPORTING TSE REALIZAT...
23620,285,63709,td/1999/l0076/article[1]/bdy[1]/sec[1],1494,2.279939,operating system,process,4.025588,INTRODUCTION wormhole-routing switching techni...
23621,285,18209,cg/1998/g4030/article[1]/bdy[1]/sec[1],1495,2.279900,operating system,process,3.670078,Spreadsheets proven highly successful interact...


1.INTENT AGNOSTIC FEATURES

In [ ]:
# calculate the sum, min, max, mean, and variance of the 'score_tfidf' for each query (grouped by 'qid')
tfidf_stats = combined_results.groupby('qid')['score_tfidf'].agg(
    sum_tfidf='sum',
    min_tfidf='min',
    max_tfidf='max',
    mean_tfidf='mean',
    var_tfidf='var'
).reset_index()

tfidf_stats

,qid,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf
0,203,8285.848930,4.608841,12.755553,5.843335,1.350430
1,209,13702.385366,6.820562,28.316350,9.515545,13.678956
2,212,11412.855782,5.512042,22.586892,8.504363,12.223822
3,218,8806.178793,5.265130,24.846264,7.381541,7.005565
4,221,12265.245119,6.823907,17.899407,8.985528,3.525984
5,241,6979.201364,3.463653,17.868735,4.819890,1.923948
6,242,7812.832616,5.097901,9.618657,6.127712,0.630656
7,248,6773.442261,3.892639,5.113783,4.515628,0.111711
8,249,6773.442261,3.892639,5.113783,4.515628,0.111711
9,253,9366.431187,5.407440,14.117156,6.802056,1.805040


In [ ]:
combined_results = pd.merge(combined_results, tfidf_stats, on='qid')

In [ ]:
# Download stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
combined_results['query'] = combined_results['query'].astype(str)
combined_results['text'] = combined_results['text'].astype(str)

In [ ]:
stop_words = set(stopwords.words('english'))

# Preprocess the text (tokenization, stopword removal)
def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Apply preprocessing to both queries and documents
combined_results['query_tokens'] = combined_results['query'].apply(preprocess)
combined_results['document_tokens'] = combined_results['text'].apply(preprocess)

# Combine queries and documents into one list for further processing
combined_text = pd.concat([combined_results['query_tokens'].apply(' '.join), combined_results['document_tokens'].apply(' '.join)], axis=0)


In [ ]:
combined_results['document_length'] = combined_results['text'].str.split().str.len()

In [ ]:
def all_words_in_text(query, text):
    query_words = set(query.lower().split())  # Tokenizing query and converting to lowercase
    text_words = set(text.lower().split())    # Tokenizing text and converting to lowercase
    return 1 if query_words.issubset(text_words) else 0  # Return 1 if all query words are found, else 0


combined_results['query_in_text'] = combined_results.apply(lambda row: all_words_in_text(row['query'], row['text']), axis=1)


2.INTENT-AWARE FEATURES

In [ ]:
import re

# Check for any of the target words (case-insensitive)
pattern = r'\b(methodology|experiment|setup)\b'

combined_results['contains_methodology'] = combined_results['text'].str.contains(pattern, case=False, regex=True).astype(int)


<ipython-input-44-cbcd49d13cf8>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  combined_results['contains_methodology'] = combined_results['text'].str.contains(pattern, case=False, regex=True).astype(int)


In [ ]:

def query_in_methodology(query, text, contains_methodology):
    if contains_methodology == 1:
        query_words = set(query.lower().split())
        text_words = set(text.lower().split())
        return 1 if query_words.issubset(text_words) else 0  # Return 1 if match found, else 0
    return 0  # If 'contains_methodology' is not 1, return 0

# Apply the function to create the new column 'query_in_methodology'
combined_results['query_in_methodology'] = combined_results.apply(lambda row: query_in_methodology(row['query'], row['text'], row['contains_methodology']), axis=1)

In [ ]:
combined_results

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,query_tokens,document_tokens,document_length,query_in_text,contains_methodology,query_in_methodology
0,203,23601,co/2000/r7108/article[1]/bdy[1]/sec[2],0,22.160864,code signing verification,process,12.755553,"CODE SIGNING VERIFICATION Typically, software ...",8285.848930,4.608841,12.755553,5.843335,1.350430,"[code, signing, verification]","[code, signing, verification, typically, softw...",199,1,0,0
1,203,11887,dt/2001/d4006/article[1]/bdy[1]/sec[3],1,21.497232,code signing verification,process,12.350038,RTL STATIC SIGN-OFF ISSUES Using concepts fram...,8285.848930,4.608841,12.755553,5.843335,1.350430,"[code, signing, verification]","[rtl, static, issues, using, concepts, framewo...",491,0,1,0
2,203,11888,dt/2001/d4006/article[1]/bdy[1]/sec[4],2,21.165475,code signing verification,process,12.153532,EFFECTIVE EQUIVALENCE-CHECKING METHODOLOGY Tod...,8285.848930,4.608841,12.755553,5.843335,1.350430,"[code, signing, verification]","[effective, methodology, today, equivalence, c...",579,0,1,0
3,203,24890,co/1997/r6076/article[1]/bdy[1]/sec[4],3,20.580498,code signing verification,process,11.867551,CONCLUSION general-purpose secure code distrib...,8285.848930,4.608841,12.755553,5.843335,1.350430,"[code, signing, verification]","[conclusion, secure, code, distribution, schem...",151,0,0,0
4,203,25503,co/1997/r6026/article[1]/bdy[1]/sec[6],4,20.172168,code signing verification,process,11.672510,"Secure Code Distribution, pp. 76-79 X. Nick Zh...",8285.848930,4.608841,12.755553,5.843335,1.350430,"[code, signing, verification]","[secure, code, distribution, pp, nick, zheng, ...",108,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23618,285,24430,co/2000/r5012/article[1]/bdy[1]/sec[1],1491,2.280486,operating system,process,3.790378,"companies adopt e-commerce, extranets, intrane...",655.326092,3.605801,4.172980,3.854859,0.022442,"[operating, system]","[companies, adopt, extranets, intranets, move,...",208,1,0,0
23619,285,48529,tk/1997/k0600/article[1]/bdy[1]/sec[5],1492,2.280477,operating system,process,3.689152,MULTIVIEW: VIEW SYSTEM SUPPORTING TSE REALIZAT...,655.326092,3.605801,4.172980,3.854859,0.022442,"[operating, system]","[multiview, view, system, supporting, tse, rea...",1292,0,0,0
23620,285,63709,td/1999/l0076/article[1]/bdy[1]/sec[1],1494,2.279939,operating system,process,4.025588,INTRODUCTION wormhole-routing switching techni...,655.326092,3.605801,4.172980,3.854859,0.022442,"[operating, system]","[introduction, switching, technique, becoming,...",539,0,0,0
23621,285,18209,cg/1998/g4030/article[1]/bdy[1]/sec[1],1495,2.279900,operating system,process,3.670078,Spreadsheets proven highly successful interact...,655.326092,3.605801,4.172980,3.854859,0.022442,"[operating, system]","[spreadsheets, proven, highly, successful, int...",462,0,0,0


In [ ]:
# TFIDF cosine similarity
combined_results['query_clean'] = combined_results['query_tokens'].apply(' '.join)
combined_results['document_clean'] = combined_results['document_tokens'].apply(' '.join)

# Combine cleaned queries and documents
combined_clean_text = pd.concat([combined_results['query_clean'], combined_results['document_clean']], axis=0)

# Apply TFIDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_clean_text)

# Split TFIDF vectors for queries and documents
query_tfidf = tfidf_matrix[:len(combined_results['query'])]  # TFIDF vectors for queries
doc_tfidf = tfidf_matrix[len(combined_results['query']):]    # TFIDF vectors for documents

# Compute cosine similarity between each query and its corresponding document
combined_results['tfidf_cosine_similarity'] = cosine_similarity(query_tfidf, doc_tfidf).diagonal()


In [ ]:
# model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Compute BERT embeddings for queries and documents
combined_results['query_embedding'] = combined_results['query'].apply(lambda x: model.encode(x, convert_to_tensor=False))
combined_results['document_embedding'] = combined_results['text'].apply(lambda x: model.encode(x, convert_to_tensor=False))

# Compute cosine similarity between each query and document using BERT embeddings
combined_results['bert_cosine_similarity'] = combined_results.apply(lambda row: cosine_similarity([row['query_embedding']], [row['document_embedding']])[0][0], axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
combined_results

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,...,document_length,query_in_text,contains_methodology,query_in_methodology,query_clean,document_clean,tfidf_cosine_similarity,query_embedding,document_embedding,bert_cosine_similarity
0,203,23601,co/2000/r7108/article[1]/bdy[1]/sec[2],0,22.160864,code signing verification,process,12.755553,"CODE SIGNING VERIFICATION Typically, software ...",8285.848930,...,199,1,0,0,code signing verification,code signing verification typically software d...,0.153413,"[-0.15310854, 0.06115929, -0.7489768, -0.53346...","[-0.30395347, -0.037272967, -0.5363751, -0.402...",0.515524
1,203,11887,dt/2001/d4006/article[1]/bdy[1]/sec[3],1,21.497232,code signing verification,process,12.350038,RTL STATIC SIGN-OFF ISSUES Using concepts fram...,8285.848930,...,491,0,1,0,code signing verification,rtl static issues using concepts framework pro...,0.042411,"[-0.15310854, 0.06115929, -0.7489768, -0.53346...","[-0.40124834, 0.113038994, 0.055635817, -0.321...",0.379981
2,203,11888,dt/2001/d4006/article[1]/bdy[1]/sec[4],2,21.165475,code signing verification,process,12.153532,EFFECTIVE EQUIVALENCE-CHECKING METHODOLOGY Tod...,8285.848930,...,579,0,1,0,code signing verification,effective methodology today equivalence checki...,0.087823,"[-0.15310854, 0.06115929, -0.7489768, -0.53346...","[-0.35692817, 0.017755393, -0.071665786, -0.43...",0.249259
3,203,24890,co/1997/r6076/article[1]/bdy[1]/sec[4],3,20.580498,code signing verification,process,11.867551,CONCLUSION general-purpose secure code distrib...,8285.848930,...,151,0,0,0,code signing verification,conclusion secure code distribution scheme mus...,0.092146,"[-0.15310854, 0.06115929, -0.7489768, -0.53346...","[-0.10944544, 0.015337421, -0.25893632, -0.403...",0.419163
4,203,25503,co/1997/r6026/article[1]/bdy[1]/sec[6],4,20.172168,code signing verification,process,11.672510,"Secure Code Distribution, pp. 76-79 X. Nick Zh...",8285.848930,...,108,0,0,0,code signing verification,secure code distribution pp nick zheng java vi...,0.136160,"[-0.15310854, 0.06115929, -0.7489768, -0.53346...","[-0.31216645, 0.1595561, -0.17923579, -0.51759...",0.405383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23618,285,24430,co/2000/r5012/article[1]/bdy[1]/sec[1],1491,2.280486,operating system,process,3.790378,"companies adopt e-commerce, extranets, intrane...",655.326092,...,208,1,0,0,operating system,companies adopt extranets intranets move opera...,0.103409,"[-0.43514195, 0.05775259, 0.2877332, -0.437219...","[0.058148067, -0.1604442, 0.043968126, -0.2961...",0.375442
23619,285,48529,tk/1997/k0600/article[1]/bdy[1]/sec[5],1492,2.280477,operating system,process,3.689152,MULTIVIEW: VIEW SYSTEM SUPPORTING TSE REALIZAT...,655.326092,...,1292,0,0,0,operating system,multiview view system supporting tse realizati...,0.022632,"[-0.43514195, 0.05775259, 0.2877332, -0.437219...","[-0.22367722, -0.12241844, -0.23226699, -0.453...",0.103811
23620,285,63709,td/1999/l0076/article[1]/bdy[1]/sec[1],1494,2.279939,operating system,process,4.025588,INTRODUCTION wormhole-routing switching techni...,655.326092,...,539,0,0,0,operating system,introduction switching technique becoming tren...,0.005104,"[-0.43514195, 0.05775259, 0.2877332, -0.437219...","[-0.4473447, -0.48026007, -0.20235434, -0.4924...",0.220769
23621,285,18209,cg/1998/g4030/article[1]/bdy[1]/sec[1],1495,2.279900,operating system,process,3.670078,Spreadsheets proven highly successful interact...,655.326092,...,462,0,0,0,operating system,spreadsheets proven highly successful interact...,0.011527,"[-0.43514195, 0.05775259, 0.2877332, -0.437219...","[-0.57061636, 0.009246323, -0.1125913, -0.3327...",0.045667


In [ ]:
combined_results = combined_results.drop(columns=['query_embedding', 'document_embedding', 'query_tokens', 'document_tokens', 'query_clean', 'document_clean'])

In [ ]:
combined_results

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,document_length,query_in_text,contains_methodology,query_in_methodology,tfidf_cosine_similarity,bert_cosine_similarity
0,203,23601,co/2000/r7108/article[1]/bdy[1]/sec[2],0,22.160864,code signing verification,process,12.755553,"CODE SIGNING VERIFICATION Typically, software ...",8285.848930,4.608841,12.755553,5.843335,1.350430,199,1,0,0,0.153413,0.515524
1,203,11887,dt/2001/d4006/article[1]/bdy[1]/sec[3],1,21.497232,code signing verification,process,12.350038,RTL STATIC SIGN-OFF ISSUES Using concepts fram...,8285.848930,4.608841,12.755553,5.843335,1.350430,491,0,1,0,0.042411,0.379981
2,203,11888,dt/2001/d4006/article[1]/bdy[1]/sec[4],2,21.165475,code signing verification,process,12.153532,EFFECTIVE EQUIVALENCE-CHECKING METHODOLOGY Tod...,8285.848930,4.608841,12.755553,5.843335,1.350430,579,0,1,0,0.087823,0.249259
3,203,24890,co/1997/r6076/article[1]/bdy[1]/sec[4],3,20.580498,code signing verification,process,11.867551,CONCLUSION general-purpose secure code distrib...,8285.848930,4.608841,12.755553,5.843335,1.350430,151,0,0,0,0.092146,0.419163
4,203,25503,co/1997/r6026/article[1]/bdy[1]/sec[6],4,20.172168,code signing verification,process,11.672510,"Secure Code Distribution, pp. 76-79 X. Nick Zh...",8285.848930,4.608841,12.755553,5.843335,1.350430,108,0,0,0,0.136160,0.405383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23618,285,24430,co/2000/r5012/article[1]/bdy[1]/sec[1],1491,2.280486,operating system,process,3.790378,"companies adopt e-commerce, extranets, intrane...",655.326092,3.605801,4.172980,3.854859,0.022442,208,1,0,0,0.103409,0.375442
23619,285,48529,tk/1997/k0600/article[1]/bdy[1]/sec[5],1492,2.280477,operating system,process,3.689152,MULTIVIEW: VIEW SYSTEM SUPPORTING TSE REALIZAT...,655.326092,3.605801,4.172980,3.854859,0.022442,1292,0,0,0,0.022632,0.103811
23620,285,63709,td/1999/l0076/article[1]/bdy[1]/sec[1],1494,2.279939,operating system,process,4.025588,INTRODUCTION wormhole-routing switching techni...,655.326092,3.605801,4.172980,3.854859,0.022442,539,0,0,0,0.005104,0.220769
23621,285,18209,cg/1998/g4030/article[1]/bdy[1]/sec[1],1495,2.279900,operating system,process,3.670078,Spreadsheets proven highly successful interact...,655.326092,3.605801,4.172980,3.854859,0.022442,462,0,0,0,0.011527,0.045667


In [ ]:
combined_results.to_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_proc_2005_features_and_sections.tsv', sep='\t', index=False)